<a href="https://colab.research.google.com/github/danwaters/lfi/blob/main/Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook retrieves the tokenized word embeddings so that I don't have to dynamically calculate them on every single prediction. These embeddings are stored in the Cloud Function that hosts the prediction.

In [12]:
!git clone https://github.com/danwaters/lfi.git

Cloning into 'lfi'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 49 (delta 17), reused 48 (delta 16), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [26]:
import tensorflow as tf
from keras import preprocessing 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

In [27]:
artists = ['beatles', 'billie-eilish', 'bob-dylan', 'eminem', 'michael-jackson', 'nickelback', 'zappa']
t = Tokenizer()

In [30]:
def get_tokens(embedding_path, seed_text):
  embedding = np.load(embedding_path)
  token_list = t.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], 
                             maxlen=max_sequence_length - 1,
                             padding='pre')
  return token_list  

In [31]:
for artist in artists:
  path = f'lfi/train/{artist}.txt'

  text = open(path, 'rb').read().decode(encoding='utf-8')

   # kill the newlines and lowercase everything
  sentences = text.lower().replace("\r\n", "\n").split("\n")

  # fit the tokenizer
  t.fit_on_texts(sentences)
  
  total_words = len(t.word_index) + 1
  print(f"There are {total_words} unique words in {artist}.txt")

  # Create input sequences, using list of tokens
  input_sequences = []
  for sentence in sentences:
    word_list = t.texts_to_sequences([sentence])[0]
    for i in range(1, len(word_list)):
      n_gram_sequence = word_list[:i+1] # include next
      input_sequences.append(n_gram_sequence)

  # Pad the sequences to fill in zeroes to maximum dimensionality 
  max_sequence_length = max([len(x) for x in input_sequences]) 
  input_sequences = np.array(pad_sequences(input_sequences, 
                                           maxlen=max_sequence_length, 
                                           padding='pre'))
  
  path = f'embeddings/{artist}.npy'
  np.save(path, input_sequences)
  print(f'Embeddings saved to {path}.')

  # Test load and tokenize embedding
  print(f'Artist embeddings: {artist}')
  token_list = get_tokens(f'embeddings/{artist}.npy', "I")
  print(token_list)

There are 11175 unique words in beatles.txt
Embeddings saved to embeddings/beatles.npy.
Artist embeddings: beatles
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]]
There are 11175 unique words in billie-eilish.txt
Embeddings saved to embeddings/billie-eilish.npy.
Artist embeddings: billie-eilish
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]]
There are 11175 unique words in bob-dylan.txt
Embeddings saved to embeddings/bob-dylan.npy.
Artist embeddings: bob-dylan
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]]
There are 11175 unique words in eminem.txt
Embeddings saved to embeddings/eminem.npy.
Artist embeddings: eminem
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]]
There are 11175 unique words in michael-jackson.txt
Embeddings saved to embeddings/michael-jackson.npy.
Artist embeddings: michael-jackson
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3]]
There are 11175 unique words in nickelback.txt
Embeddings saved to embeddings/nickelback.npy.
Artist embeddings: nickelback


In [32]:
artist_embeddings = {
    "beatles": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]],
    # "billie-eilish": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]],
    "bob-dylan": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]],
    # "eminem": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]],
    "michael-jackson": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]],
    # "nickelback": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]],
    "zappa": [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]]
}

In [34]:
# Test with an array of predictions - Beatles
predictions = [[1.08432441e-09, 0.000144436053, 0.00234247651, 6.57938e-07, 7.58562164e-05, 0.00236697611, 5.25136024e-09, 7.62923094e-07, 0.0372449532, 0.000675282034, 1.76756066e-06, 9.21519e-06, 7.75597631e-09, 3.97043401e-13, 1.00668024e-06, 3.37616484e-07, 5.42651313e-10, 2.62400681e-05, 0.000199240589, 5.01002262e-09, 3.61901975e-09, 3.57185831e-06, 0.000491847692, 1.374466e-13, 0.000114336799, 0.000779119437, 0.000304754532, 4.53162975e-05, 9.43913449e-07, 1.57515112e-09, 8.05040108e-05, 0.00018141685, 2.9029763e-09, 6.00189651e-06, 0.011386944, 2.16949e-05, 9.40076643e-05, 8.88055e-07, 1.31161304e-08, 2.38386519e-08, 0.000733933586, 8.00149064e-05, 2.75408615e-06, 2.63653283e-06, 2.55229726e-10, 5.88815247e-11, 1.82689974e-10, 1.67411542e-06, 2.47916591e-06, 1.70383716e-12, 0.000151010376, 4.14615e-05, 3.8908976e-10, 1.03741987e-10, 1.59411731e-12, 3.49737311e-05, 3.48782705e-05, 0.00727671245, 8.09777643e-15, 1.53245965e-05, 0.000961594225, 0.000131580266, 1.80080676e-07, 0.000163883335, 6.03813751e-05, 5.14520934e-07, 1.95139705e-07, 4.96623443e-07, 0.000998234376, 5.39755698e-18, 7.21316284e-10, 4.56049226e-10, 1.21716708e-15, 5.53818484e-11, 4.5952282e-09, 0.0942207053, 6.18976337e-05, 0.0118552139, 9.79036486e-05, 4.37625225e-08, 0.0283036251, 6.87744528e-09, 3.68718632e-11, 3.36672292e-12, 2.5021011e-06, 4.03162881e-09, 3.90047322e-15, 0.000947271357, 0.000173742606, 0.000148163599, 0.0503007136, 4.47203519e-07, 1.28627015e-17, 0.00066264288, 4.12577457e-08, 0.103324801, 1.21856087e-15, 9.82329311e-07, 0.000108771819, 4.60463207e-06, 0.00117131812, 7.67420199e-13, 1.33960009e-06, 4.83664962e-07, 0.00935100764, 0.0265288688, 2.11469758e-18, 7.15595668e-16, 3.80529563e-08, 8.12289363e-05, 0.00404568389, 4.32292363e-05, 0.00225566537, 0.00449261395, 0.00128054863, 9.71392744e-10, 4.80404765e-07, 0.0010489159, 0.000254629529, 6.12098884e-05, 1.4184684e-07, 0.00189321418, 7.33849229e-05, 3.94630515e-05, 1.88754439e-05, 0.00337061263, 1.73269642e-16, 5.76443338e-09, 7.99741162e-10, 4.14110727e-05, 3.38114873e-17, 3.05373611e-08, 3.8274186e-07, 1.73698014e-10, 5.33549418e-08, 7.09038575e-17, 6.91083174e-24, 0.000295415375, 1.83408861e-06, 4.29332486e-13, 4.24559311e-11, 8.53688e-09, 0.0229968186, 2.71928329e-08, 0.00118029315, 0.000928048044, 5.30202487e-05, 0.00304937852, 4.42542796e-06, 6.21544591e-13, 3.7549953e-06, 5.32581794e-07, 3.12827979e-06, 2.71203365e-18, 0.000990978209, 5.96788468e-08, 2.51309689e-06, 0.000742679869, 0.000546733849, 0.000315081037, 4.06450117e-11, 2.56177718e-11, 4.41714609e-10, 2.69515816e-20, 1.02519987e-15, 1.46636338e-13, 1.74440116e-17, 1.29271269e-19, 0.00774198771, 6.65745584e-11, 0.000103459926, 5.20496066e-24, 7.44823578e-11, 6.85477335e-06, 1.86462344e-07, 9.22685e-06, 8.07848e-07, 5.10864544e-15, 3.42393905e-15, 1.73356162e-17, 3.80067843e-21, 9.85019271e-12, 1.25858235e-09, 1.91891659e-11, 0.000281235902, 4.38985702e-18, 3.30248986e-06, 1.46647194e-10, 1.72296259e-22, 8.37364178e-10, 3.26568506e-09, 9.93837124e-10, 6.53342056e-08, 9.74145244e-12, 6.65736044e-09, 1.35068561e-07, 5.22407835e-16, 9.39412e-08, 5.58519e-18, 5.16423282e-15, 9.22375173e-14, 0.00919289142, 2.19774199e-11, 6.99269364e-13, 1.0412266e-20, 0.000349341484, 2.6209937e-14, 7.11507823e-07, 1.07837459e-12, 3.51182656e-08, 1.27407546e-07, 1.18354588e-11, 9.82937065e-10, 9.01127062e-10, 5.44354763e-13, 4.07293861e-14, 0.0377272069, 1.6307722e-20, 8.12610836e-07, 0.00173307804, 4.92775543e-11, 4.01891157e-07, 7.93834514e-16, 1.1959724e-11, 1.84963277e-11, 7.85873442e-14, 3.29099094e-05, 4.74203775e-20, 5.44779723e-05, 1.09694351e-08, 1.23289283e-06, 4.15090085e-07, 1.07150508e-25, 3.79315643e-05, 0.000699739554, 1.30504695e-05, 2.36442659e-07, 7.26515907e-07, 7.29941038e-11, 8.25474e-10, 9.09992144e-15, 0.000297915831, 1.0508402e-07, 5.77855404e-25, 8.48242775e-16, 1.62981428e-09, 0.00326252612, 0.000249614532, 1.93784802e-18, 3.51977633e-06, 5.15568109e-11, 1.18454926e-10, 3.09653774e-06, 1.54558535e-07, 2.22737231e-06, 9.73912173e-10, 1.31400923e-12, 1.67171899e-21, 1.49070889e-09, 3.22498352e-07, 1.21283311e-12, 2.7027045e-09, 1.19032972e-09, 2.09696792e-15, 1.85231918e-15, 2.22689573e-18, 1.04807327e-08, 7.45895845e-10, 0.000250855, 0.000666903157, 1.60487526e-11, 6.47882814e-09, 4.27567193e-06, 1.92313232e-09, 2.65130075e-19, 5.28083e-09, 0.00260152947, 0.0223329, 6.51984e-15, 0.00823891442, 9.14629691e-11, 5.93390439e-07, 2.78461791e-20, 2.31220831e-13, 2.78218162e-16, 3.63309695e-11, 4.91874955e-15, 1.42335847e-17, 6.8847432e-05, 1.68208534e-17, 1.02518361e-05, 2.17046457e-22, 0.00270204269, 0.000226562464, 1.95369929e-10, 9.95999217e-05, 9.2911856e-10, 2.59551275e-14, 1.24885901e-06, 9.53709778e-10, 4.83026597e-05, 5.9855256e-06, 0.00100684492, 1.30719616e-10, 3.30271643e-10, 6.22939876e-18, 0.00517495302, 2.45609909e-17, 7.39522918e-13, 3.59292831e-26, 6.44877481e-15, 0.000127467516, 8.15150042e-23, 4.09292114e-14, 6.77722489e-10, 1.85640759e-19, 0.000356687146, 8.3227262e-20, 2.87494434e-14, 1.45393192e-12, 1.93311469e-15, 1.23238271e-06, 9.10800269e-10, 1.01237015e-06, 8.47668489e-05, 2.04241582e-14, 5.36468724e-05, 1.04859622e-21, 3.58182733e-05, 3.19385712e-10, 9.78863142e-08, 6.27237845e-11, 5.59949531e-10, 0.00014513942, 7.70192946e-06, 6.97125255e-17, 0.000123563208, 1.92751177e-18, 0.00547978561, 9.83674947e-12, 0.000420698, 4.38286717e-16, 8.30201952e-09, 2.15688549e-16, 8.46447392e-08, 1.63959504e-13, 5.40667955e-09, 3.46481772e-17, 0.000180174393, 3.74363901e-10, 1.38978703e-06, 2.03684617e-13, 0.00904027559, 0.000323096669, 0.000121565659, 1.13625732e-22, 2.41687452e-08, 0.00916166417, 5.8129e-10, 8.8379537e-10, 9.13643328e-10, 5.94323375e-12, 0.000774904562, 2.23832487e-07, 1.97322347e-16, 2.0641183e-08, 1.33886506e-05, 1.00710634e-10, 2.40782109e-14, 9.26902444e-10, 1.93640744e-14, 1.23095269e-05, 1.74083833e-24, 1.39188316e-13, 0.0040938342, 0.00124733732, 1.18009824e-09, 1.25955753e-20, 4.08819204e-07, 3.77858552e-12, 8.56179661e-09, 7.01781567e-16, 8.00229202e-07, 1.39096548e-16, 2.94178678e-17, 4.02795558e-05, 1.42349376e-14, 3.0394e-12, 1.39610233e-06, 5.14097133e-11, 9.39403549e-07, 0.000178750459, 1.81279828e-12, 2.49742e-09, 3.47617652e-06, 1.99998317e-11, 1.23876265e-09, 1.21654117e-10, 2.81449229e-17, 2.08971529e-09, 1.01570685e-09, 1.0518509e-33, 2.74365576e-15, 1.22198429e-08, 2.87263589e-07, 9.38447653e-10, 9.33306654e-10, 1.46383242e-12, 1.34290387e-12, 1.02556946e-34, 8.87308671e-10, 0.000642446626, 1.48969887e-14, 7.0490463e-16, 1.82129976e-27, 8.44959826e-20, 0.00419782847, 0.000118063253, 2.23301311e-17, 2.45188939e-11, 1.35066974e-15, 9.55781159e-23, 8.63362715e-08, 2.12936535e-18, 1.54770561e-08, 6.51020416e-13, 2.4304028e-25, 1.6238966e-13, 5.52112579e-05, 4.85653413e-18, 3.85013863e-18, 1.74611713e-15, 1.70456793e-09, 3.9855675e-07, 0.000561237568, 1.13003686e-11, 2.38030068e-10, 8.72077699e-09, 4.89535111e-15, 6.78943593e-07, 4.58296725e-12, 1.35578368e-10, 6.17806206e-09, 1.13291342e-19, 4.10570408e-07, 1.26650875e-07, 6.6952538e-10, 4.87734434e-19, 0.00137281721, 5.21455641e-14, 9.09726197e-16, 1.39893556e-14, 1.74730772e-10, 5.31214619e-28, 5.83202745e-05, 0.000155353526, 0.000360670063, 1.01644315e-09, 1.98569775e-16, 3.71006301e-08, 9.55780342e-06, 9.37220412e-10, 3.56430488e-13, 1.3943546e-05, 1.25074792e-27, 5.37586089e-13, 1.40675485e-24, 0.000113692455, 0.0468352772, 4.23017038e-11, 0.000116729621, 2.67023491e-12, 0.000118045697, 0.0310680252, 2.00334505e-09, 7.79074541e-16, 0.000113952752, 0.000629887218, 2.6342013e-09, 0.00064187462, 0.000149145766, 8.93754304e-07, 3.59120698e-12, 5.67282683e-08, 2.68523584e-19, 8.09215e-09, 2.55490827e-21, 2.93897e-18, 4.61307354e-05, 1.42320559e-07, 8.06627795e-05, 0.000777577, 8.80560111e-13, 1.62949104e-20, 0.000468932412, 3.92982257e-14, 0.0116526205, 2.50387276e-13, 3.26071178e-24, 2.3957516e-09, 5.80893511e-09, 2.56136934e-09, 1.25321634e-28, 7.90343416e-19, 1.06815723e-09, 8.83738138e-10, 0.00382591528, 1.01740139e-09, 3.54334028e-20, 1.12249307e-10, 7.75448279e-05, 9.36187794e-10, 0.00100481906, 1.01516393e-10, 5.00206107e-27, 1.71330083e-12, 6.39646713e-15, 3.55365112e-11, 3.47121151e-11, 2.65229551e-24, 6.39808666e-16, 8.10060796e-10, 3.23496556e-06, 4.24538671e-09, 2.59035506e-21, 1.05205433e-09, 1.49925327e-07, 4.19876733e-06, 1.09998032e-10, 2.08595744e-19, 8.52936209e-05, 3.10324623e-15, 9.02423969e-10, 0.000162393815, 1.55316447e-27, 4.38302461e-09, 1.12742988e-07, 2.81583148e-08, 1.34004099e-19, 7.08541961e-07, 6.67657726e-12, 5.32180509e-15, 2.39773072e-13, 1.92622577e-14, 0.00148857955, 1.63710479e-09, 1.90835024e-07, 1.29011232e-14, 8.2279461e-12, 1.36414713e-28, 9.89922455e-07, 1.30289768e-13, 1.82224675e-25, 1.34541647e-23, 5.05606057e-10, 4.55044e-06, 4.85889828e-10, 1.13230469e-18, 1.64505366e-07, 1.38523251e-06, 1.68177543e-08, 1.38878416e-19, 5.46249261e-26, 1.78970749e-05, 1.12725673e-09, 2.21379406e-17, 4.73074246e-10, 0.00106199097, 4.21222275e-08, 3.63205345e-06, 2.8316724e-11, 8.70752445e-14, 1.2751632e-09, 1.91084924e-14, 6.96037228e-09, 1.56817568e-05, 3.33528752e-17, 8.30834836e-07, 1.28264849e-18, 2.68778513e-06, 2.53422809e-06, 2.02481608e-12, 1.90696365e-11, 3.3938366e-33, 3.75552361e-07, 2.16354683e-21, 6.53440924e-14, 6.76394635e-11, 1.57161437e-11, 9.04736286e-11, 5.93430701e-16, 4.15113104e-18, 9.0680996e-10, 1.00537956e-09, 8.15762278e-17, 2.45774635e-15, 8.77309114e-10, 2.41092233e-11, 8.76192063e-10, 8.87108942e-10, 5.64430496e-18, 9.92170346e-10, 2.02849274e-34, 2.09475479e-06, 9.50459267e-10, 7.16727605e-21, 8.59287641e-10, 1.36536407e-11, 9.31141e-10, 1.03452757e-09, 9.0317126e-10, 9.98622121e-28, 7.9854668e-05, 8.69569194e-10, 8.47060255e-10, 1.7055898e-08, 1.75605135e-06, 4.99523267e-09, 0.000139567244, 9.32626865e-10, 7.52510576e-14, 4.23163761e-12, 3.43981803e-08, 1.02704856e-09, 3.56974758e-13, 8.5131685e-10, 1.02859792e-08, 0.0213195123, 6.1217087e-10, 1.19193629e-16, 1.83660269e-24, 9.60149071e-10, 8.78816464e-10, 9.66742353e-10, 3.7908943e-19, 0.0127897495, 1.57881463e-09, 1.25503244e-16, 1.17501331e-09, 2.26498376e-09, 9.94478e-10, 9.37671e-10, 8.29935454e-10, 3.65800229e-23, 1.62627511e-08, 1.03677958e-05, 2.61109667e-12, 0.000139005628, 1.30747173e-08, 3.80450202e-13, 0.00138002622, 3.08076278e-20, 5.64137834e-14, 5.61665365e-06, 3.63675791e-11, 6.54706155e-32, 2.3943312e-05, 2.42335918e-05, 3.3246033e-14, 2.55015884e-08, 2.041089e-28, 1.8805828e-21, 3.59532748e-15, 1.12955843e-11, 2.03751838e-09, 1.09609507e-18, 1.08257905e-15, 6.95451943e-13, 0.00237053167, 3.97611903e-16, 0.00136496336, 6.7206826e-12, 7.4447039e-06, 5.05946676e-17, 5.32137904e-15, 1.77307868e-09, 2.6351826e-23, 2.10906705e-06, 2.48656164e-07, 4.5766359e-15, 7.92976048e-21, 1.03484556e-16, 9.30462818e-10, 3.14926609e-08, 8.75387e-10, 0.00150095019, 7.19618778e-15, 4.44454265e-11, 1.49182462e-08, 8.69530854e-18, 1.25186981e-08, 4.49446925e-05, 0.015564315, 9.01605e-10, 5.53691745e-11, 1.88548971e-20, 1.00964244e-19, 0.00287798862, 0.000757359725, 3.71251668e-13, 9.62374846e-10, 1.58055485e-07, 6.61971563e-13, 2.45437159e-09, 7.32171322e-13, 1.19474935e-05, 0.0295050256, 1.53383511e-15, 2.44455552e-13, 6.98769098e-10, 6.70004883e-05, 2.52579275e-06, 8.99856389e-28, 8.71396821e-07, 1.91293092e-11, 1.73091845e-17, 4.8086965e-14, 5.9637273e-28, 6.49762415e-17, 2.1294029e-06, 2.24928527e-07, 1.67815028e-17, 1.07596376e-21, 6.60434329e-09, 1.89005393e-29, 1.28851811e-19, 1.07956621e-08, 1.42998203e-13, 0.00038389102, 1.13677789e-10, 9.04415876e-10, 0.00244617253, 7.64893315e-10, 1.34873517e-13, 2.0440277e-08, 1.73216904e-07, 3.93255277e-06, 2.36613396e-09, 2.16063718e-08, 2.39359779e-15, 1.73560656e-13, 1.65528979e-10, 2.2808207e-16, 3.62573527e-09, 8.82229123e-10, 9.4384367e-10, 9.59274e-10, 9.07713293e-10, 8.94486263e-10, 8.15318757e-10, 9.87025128e-10, 9.66388414e-10, 2.30248248e-13, 8.72110495e-10, 8.021519e-10, 1.00964149e-21, 1.89504044e-05, 8.46627379e-10, 1.037741e-09, 1.00536424e-09, 9.68403358e-10, 9.43904843e-10, 8.95838459e-10, 1.0171104e-09, 5.83657584e-06, 1.58467449e-19, 0.000244439434, 5.74464608e-12, 9.50089341e-10, 9.08759568e-10, 7.73104e-10, 8.13830822e-20, 1.10521551e-05, 9.26435817e-10, 8.68554728e-10, 8.68389083e-10, 9.39135103e-10, 9.44437861e-10, 9.49756e-10, 8.76904327e-10, 9.68281455e-10, 8.80910844e-10, 8.76727e-10, 6.1734221e-20, 8.6843871e-10, 1.95795533e-18, 1.65017583e-11, 3.89303173e-12, 0.0, 4.40116346e-12, 9.12027509e-10, 9.76151271e-10, 9.61820623e-10, 9.4797159e-10, 9.92655e-10, 9.44978429e-10, 8.57712401e-10, 8.61608229e-10, 9.30846233e-10, 2.98036422e-11, 1.30184655e-10, 9.10431952e-10, 5.96685246e-18, 1.01137654e-09, 1.56929693e-21, 8.91107799e-10, 9.48025769e-10, 1.78085498e-19, 9.29923472e-10, 2.00353297e-12, 5.71575793e-20, 8.46808235e-10, 8.47884651e-10, 5.62269502e-11, 6.98377844e-05, 9.82892101e-10, 6.11245863e-13, 1.40126302e-08, 3.13267e-20, 1.39312551e-14, 1.00733304e-10, 1.63572639e-15, 3.44607338e-06, 2.88790112e-08, 0.000201332106, 0.000708016159, 3.61235629e-06, 6.57785861e-08, 4.33560124e-15, 1.65656524e-22, 1.19468373e-08, 8.55761595e-09, 2.67712744e-06, 5.74319086e-15, 3.29746186e-13, 1.23610313e-12, 0.000104605235, 1.0081204e-15, 3.44589735e-09, 1.3343375e-05, 3.5069902e-06, 0.000356686476, 1.71652223e-20, 4.98103836e-05, 1.33389079e-07, 1.3694222e-11, 3.41740041e-08, 4.04483503e-18, 2.45379368e-07, 1.67439222e-11, 1.1167821e-11, 5.37244375e-07, 6.97950161e-13, 8.70149292e-30, 1.15131662e-07, 5.8247631e-18, 3.27690885e-32, 2.88458614e-05, 3.00755311e-14, 2.7564568e-17, 1.32833962e-20, 1.10004226e-12, 3.92726088e-06, 1.34281136e-05, 6.18447361e-21, 9.99193794e-09, 1.269462e-14, 0.00206387555, 1.13090914e-09, 8.51610652e-14, 6.82883958e-11, 9.14804565e-10, 8.7285934e-10, 1.86913235e-06, 3.06499137e-16, 7.32102226e-11, 6.86745e-11, 9.5261017e-13, 1.96510879e-18, 2.46700615e-09, 2.33373353e-16, 3.33774924e-10, 3.41538879e-07, 2.79489821e-20, 1.48893415e-11, 5.49818132e-14, 1.27394579e-12, 1.18210337e-05, 9.74613741e-08, 1.4321694e-19, 8.72647604e-07, 1.19663586e-08, 6.13708669e-16, 5.07094265e-14, 7.64783579e-07, 1.63654901e-09, 2.68256393e-15, 1.43936772e-17, 0.000214908825, 5.59923544e-30, 2.55102736e-06, 2.61407805e-11, 1.62589332e-12, 2.0727833e-18, 9.24316448e-24, 3.65489896e-08, 3.75312675e-06, 5.55423458e-14, 0.00167848216, 1.89315702e-10, 9.5222184e-16, 6.73009866e-23, 6.77890899e-09, 6.13575231e-31, 5.70425093e-22, 4.69827e-07, 1.41600793e-08, 1.1765164e-08, 1.30883908e-08, 4.39895871e-07, 1.27666972e-05, 1.61539572e-16, 8.97240807e-06, 0.000728919695, 0.0017901496, 0.00933625083, 0.000296659302, 1.57432532e-05, 2.14927766e-07, 2.47632488e-14, 3.09243098e-09, 3.37554375e-05, 1.84169551e-06, 1.43997704e-06, 3.92748476e-14, 1.49863032e-14, 6.13849043e-05, 6.53098759e-05, 4.79497761e-25, 8.52817166e-13, 1.43928961e-15, 0.0399279855, 9.53550732e-34, 5.40836936e-06, 1.48280492e-06, 1.50379387e-09, 3.70512288e-10, 9.35998501e-10, 1.36144081e-06, 6.6278626e-06, 8.06904336e-06, 3.9810566e-10, 1.98427453e-23, 1.08469874e-13, 1.23938877e-13, 4.48329132e-07, 2.25784333e-07, 7.21763127e-15, 8.36191793e-12, 0.000716606737, 4.69739062e-15, 8.34042385e-06, 4.25246782e-10, 1.22594783e-20, 4.53838257e-21, 9.55893215e-14, 3.79532139e-08, 1.87009704e-19, 5.43859228e-11, 1.56975784e-06, 0.000868178089, 7.85170551e-22, 1.29972505e-14, 1.99216034e-14, 1.2297136e-20, 6.60243273e-08, 9.17033727e-10, 1.05731589e-22, 1.08731031e-10, 1.72771619e-09, 9.86219766e-06, 1.65637217e-26, 4.50152814e-17, 1.23444033e-11, 4.11891646e-20, 3.23625352e-22, 2.53456117e-10, 9.16778375e-10, 7.1116591e-07, 8.81788365e-10, 9.83469528e-10, 2.75478357e-07, 3.00451692e-16, 5.19508104e-17, 6.87696676e-22, 2.10318426e-19, 5.44742532e-19, 8.75524275e-09, 6.04968302e-18, 1.715361e-23, 6.90062437e-20, 6.599854e-11, 2.92364e-14, 1.55332012e-25, 5.97285859e-20, 3.67358566e-09, 0.00100017071, 0.00170321797, 1.32693331e-25, 6.37060193e-20, 8.23072099e-09, 6.51498092e-11, 9.05300908e-07, 8.39977261e-08, 2.17263278e-24, 6.74059587e-15, 9.16337919e-21, 1.02104447e-09, 9.54194897e-15, 4.42016146e-11, 2.87569623e-21, 7.68123e-17, 9.31695276e-10, 9.66311e-10, 2.38346259e-10, 2.51390247e-06, 8.60077e-16, 3.03815284e-09, 1.38694691e-08, 2.90074031e-09, 4.69085925e-13, 2.82078232e-23, 1.19123543e-25, 1.7070892e-13, 8.5437911e-08, 1.10844383e-08, 3.88875658e-16, 4.0460394e-18, 6.11021619e-07, 3.06637207e-23, 7.78181117e-22, 8.36804584e-12, 8.49354365e-10, 9.20854781e-10, 9.34935129e-10, 9.33100153e-10, 8.81768158e-10, 9.26729193e-10, 8.59694149e-10, 9.92920079e-10, 1.00638875e-09, 9.88434223e-10, 8.94691e-10, 1.01098685e-09, 9.04512465e-10, 9.84866189e-10, 8.41649084e-10, 9.03784714e-10, 9.01233588e-10, 9.51649093e-10, 9.61556501e-10, 9.408061e-10, 8.63519922e-10, 9.35234779e-10, 9.30455657e-10, 9.92908755e-10, 9.25312604e-10, 8.30182478e-10, 9.16114129e-10, 9.46327572e-10, 8.25288282e-10, 9.5603e-10, 9.18059351e-10, 9.21184962e-10, 8.27608593e-10, 8.56679061e-10, 9.45169498e-10, 9.75425407e-10, 8.88429719e-10, 8.85303719e-10, 1.03990061e-09, 9.07470876e-10, 8.93394969e-10, 9.52237511e-10, 9.46981271e-10, 1.04872855e-09, 9.06142494e-10, 1.01356246e-09, 9.35627353e-10, 9.33288891e-10, 9.39138767e-10, 9.18230936e-10, 9.01687502e-10, 9.00024222e-10, 1.03335218e-09, 8.69234185e-10, 9.539316e-10, 8.58353943e-10, 9.69375358e-10, 1.01492437e-09, 9.45097445e-10, 9.31894339e-10, 9.32573574e-10, 9.05561959e-10, 8.19928569e-10, 9.78690462e-10, 9.75339809e-10, 8.38038472e-10, 9.20949539e-10, 9.50056811e-10, 9.60094e-10, 9.27093458e-10, 9.1730662e-10, 9.5002417e-10, 8.81441919e-10, 9.3567365e-10, 8.14886492e-10, 8.79983864e-10, 8.16504586e-10, 9.05144126e-10, 8.76279e-10, 8.64795735e-10, 8.97966645e-10, 1.02053432e-09, 1.00936459e-09, 8.47461046e-10, 8.33892788e-10, 8.87562579e-10, 8.87857177e-10, 9.330966e-10, 7.84715737e-10, 9.15541254e-10, 9.67982361e-10, 8.22977575e-10, 9.43699563e-10, 8.10669809e-10, 8.98367492e-10, 9.23651655e-10, 9.55917e-10, 9.2931346e-10, 9.41071665e-10, 9.21473231e-10, 9.36387856e-10, 8.38236647e-10, 1.00452091e-09, 9.0674418e-10, 8.46407777e-10, 9.03536523e-10, 9.13503828e-10, 9.06630104e-10, 8.81368e-10, 8.9219282e-10, 9.53091495e-10, 9.69767489e-10, 9.61817e-10, 9.12305953e-10, 8.50742199e-10, 1.04763698e-09, 9.46659862e-10, 9.83300774e-10, 9.54383e-10, 1.06969444e-09, 8.7417229e-10, 9.79852199e-10, 9.05910957e-10, 9.23123245e-10, 1.09736564e-09, 9.44182066e-10, 1.02625364e-09, 9.10713338e-10, 8.16352e-10, 1.05113973e-09, 8.51287596e-10, 8.74248951e-10, 9.40458e-10, 8.54772697e-10, 8.5374946e-10, 9.32185773e-10, 9.24201438e-10, 8.69177841e-10, 8.85269968e-10, 9.79459847e-10, 8.65010175e-10, 8.63249916e-10, 9.31851707e-10, 9.55435731e-10, 9.23644605e-10, 9.78078396e-10, 8.77339312e-10, 9.55297286e-10, 8.97209917e-10, 8.76633433e-10, 8.93354168e-10, 9.52778856e-10, 9.94068383e-10, 9.73347625e-10, 8.51089532e-10, 8.88839946e-10, 8.50145343e-10, 9.55734714e-10, 1.02646502e-09, 8.72077244e-10, 9.54099e-10, 1.05266851e-09, 8.57074578e-10, 9.48792822e-10, 8.86527129e-10, 9.71252e-10, 9.16226039e-10, 9.48876089e-10, 8.22444e-10, 8.52366455e-10, 9.069967e-10, 9.57113833e-10, 8.95353403e-10, 9.97209426e-10, 1.00708386e-09, 1.00508424e-09, 9.36423605e-10, 8.92022678e-10, 1.01016573e-09, 8.45988168e-10, 8.48412063e-10, 9.49654577e-10, 1.09944387e-09, 9.03057573e-10, 9.04567754e-10, 8.6783275e-10, 9.47046219e-10, 1.01269293e-09, 3.16971738e-09, 1.88480056e-15, 1.39779379e-18, 8.77703367e-23, 3.49585642e-14, 0.000114092814, 8.19545874e-13, 1.36209642e-12, 9.56802e-13, 7.46926201e-11, 2.09457184e-24, 1.02532385e-06, 3.30002845e-17, 4.75144338e-13, 2.44557894e-15, 1.66701278e-10, 5.16356604e-14, 2.49344079e-09, 9.61501434e-10, 6.16177165e-10, 1.58376833e-15, 9.44160528e-10, 1.43121446e-14, 3.35961484e-16, 5.92100628e-14, 1.00722986e-09, 1.53710893e-08, 7.54881953e-08, 1.7072138e-12, 9.64502297e-11, 1.05428704e-11, 2.23583704e-11, 9.54445e-10, 4.42607188e-20, 9.38827127e-10, 2.46997502e-07, 2.98570413e-09, 5.56549e-20, 4.57028568e-26, 7.87594386e-22, 2.72975566e-11, 1.81133553e-09, 5.82387429e-18, 0.0264665522, 6.57302657e-09, 1.73032987e-07, 1.24875628e-14, 3.88399436e-14, 4.66956962e-09, 1.47952346e-23, 3.86767136e-14, 2.51506511e-23, 3.5855802e-16, 1.36282698e-19, 1.29678944e-16, 2.23444085e-09, 2.09450904e-38, 9.86505655e-10, 3.18314351e-20, 5.43516512e-13, 1.00330787e-07, 6.08651078e-07, 6.69520617e-09, 6.01477622e-14, 1.01162723e-09, 6.86156132e-10, 1.76127096e-32, 1.25760389e-08, 6.31039387e-11, 4.20925126e-08, 5.63757591e-12, 2.26320113e-12, 4.84004823e-21, 7.78264223e-07, 2.26942197e-19, 7.1737305e-10, 8.78102646e-10, 4.55694638e-09, 2.15565882e-17, 2.99676363e-26, 2.2342013e-10, 9.97819466e-15, 3.61010438e-17, 1.72416855e-19, 1.41022697e-11, 7.58313869e-15, 1.77052644e-25, 2.27307966e-14, 5.37109665e-20, 8.64752404e-22, 4.88641896e-13, 9.5266e-19, 3.10423182e-17, 4.76988944e-15, 2.03559672e-23, 6.41834914e-17, 2.87871584e-20, 2.84200137e-15, 1.99021875e-11, 9.41380529e-10, 2.07503898e-17, 1.1626034e-23, 5.1103383e-20, 3.22366437e-20, 6.92345168e-07, 1.51716435e-13, 4.9515787e-09, 1.1237e-16, 2.23736685e-20, 1.95923722e-10, 2.81252396e-05, 2.4263145e-05, 2.08275687e-23, 1.5449472e-24, 1.45724229e-07, 4.49915888e-06, 1.10640352e-34, 1.44073409e-34, 5.12442389e-12, 1.57912282e-19, 7.68499312e-29, 2.69017447e-10, 1.88432108e-24, 4.13309525e-10, 2.03456578e-29, 1.20480084e-11, 1.20853016e-21, 5.12301659e-12, 7.69295719e-14, 5.24992645e-32, 3.67681041e-09, 1.69719561e-22, 5.72290756e-11, 1.43295519e-28, 1.84512727e-17, 5.16235559e-06, 0.0106615406, 9.18371e-10, 2.00425561e-08, 3.20567687e-15, 1.18279926e-14, 1.68624956e-12, 8.21319261e-08, 3.92533402e-11, 1.05669185e-09, 1.76513874e-13, 9.9426567e-10, 8.31316179e-12, 8.94254226e-10, 8.71415917e-09, 4.25352464e-09, 1.2260265e-10, 4.57215274e-18, 9.11223381e-08, 1.0848582e-09, 1.0081204e-15, 1.99656616e-13, 2.86643363e-19, 3.00073677e-16, 2.51471164e-07, 2.30107333e-09, 1.32196713e-17, 9.03548326e-18, 8.93238261e-10, 6.60361e-06, 1.74585918e-13, 9.30697075e-10, 2.86665232e-19, 6.64934324e-27, 3.36479656e-16, 2.95979422e-15, 9.63469526e-10, 1.90933869e-09, 6.44747819e-13, 2.05353578e-30, 3.1948656e-12, 4.99025192e-13, 1.00580737e-05, 1.88603202e-17, 1.60649518e-15, 7.26299629e-30, 7.85127858e-24, 1.6471374e-12, 1.81254827e-17, 1.75942298e-26, 7.28744322e-12, 1.37595269e-10, 9.76250764e-12, 8.72511698e-17, 6.22631633e-05, 4.51457794e-07, 2.32288798e-13, 1.31798012e-12, 2.564072e-13, 9.64285762e-10, 6.83584613e-12, 2.86180057e-08, 8.87123406e-08, 5.90412229e-22, 2.26431105e-08, 3.00952839e-13, 8.66208771e-10, 2.80057972e-25, 9.63991442e-10, 1.4945773e-06, 9.43613845e-20, 1.0898624e-17, 1.93180749e-10, 1.85959094e-18, 4.05831748e-12, 0.00654870691, 1.78051393e-28, 1.01685838e-20, 2.52990272e-12, 1.50225342e-14, 6.28440147e-11, 2.68747975e-08, 2.4578332e-13, 1.45897357e-20, 9.39763864e-18, 2.10861558e-07, 1.12440753e-12, 5.32874578e-10, 5.9752904e-11, 7.22420179e-11, 7.35822861e-12, 1.14782917e-09, 4.3203745e-06, 1.21085033e-15, 2.16974909e-14, 7.22751303e-10, 0.00336673832, 6.23921442e-05, 4.69328049e-11, 4.52323694e-16, 3.43233923e-16, 1.76144288e-09, 2.11007e-20, 8.45228781e-13, 3.22483743e-06, 5.29191318e-12, 1.51813083e-06, 8.27231e-26, 3.11044687e-07, 2.36225958e-17, 2.60608049e-24, 1.54002766e-17, 8.25477242e-10, 1.22718447e-14, 3.56181822e-24, 2.49152032e-09, 4.44157942e-21, 4.98122369e-22, 5.87584e-09, 1.12140185e-06, 6.34910886e-27, 1.49121931e-07, 1.4424726e-18, 1.2528981e-25, 8.62631e-10, 4.02886871e-13, 3.13727065e-11, 2.52907792e-08, 2.12498517e-13, 1.48403416e-23, 1.10338175e-26, 7.523055e-21, 1.79096909e-22, 7.78745263e-11, 0.000146785038, 9.5805619e-10, 7.13621603e-06, 1.83440589e-11, 3.52034138e-28, 5.58573521e-10, 2.70548422e-13, 8.8716115e-07, 9.70526104e-10, 1.83996766e-14, 4.19303603e-14, 8.4328114e-30, 1.3305214e-07, 3.37110032e-26, 6.13011728e-30, 2.79511e-06, 2.09932627e-10, 1.54722984e-05, 7.65798713e-09, 3.20179737e-07, 9.84152204e-11, 8.06975074e-25, 1.13052974e-05, 2.98136946e-08, 2.2681104e-12, 8.9056563e-14, 3.73242895e-11, 2.18624245e-13, 1.95134024e-08, 8.94193422e-20, 5.10457054e-09, 2.30587522e-13, 1.26679796e-23, 2.62763616e-10, 6.02399052e-23, 1.06354686e-27, 2.36610784e-08, 1.27150088e-05, 3.69953347e-27, 4.47464467e-15, 1.77135757e-06, 9.43471932e-08, 2.39860791e-14, 9.40273077e-18, 2.66940541e-19, 7.21327941e-27, 6.76863631e-17, 2.6983793e-08, 1.25463358e-26, 6.05775899e-20, 1.8984109e-25, 3.50274192e-08, 1.26822901e-12, 9.04398612e-10, 5.63086658e-17, 1.75755358e-05, 5.49435786e-09, 1.11183107e-09, 1.97247232e-07, 7.21254492e-06, 1.50014551e-26, 7.55065876e-09, 1.28293522e-21, 3.43760339e-18, 1.89172879e-05, 1.92346035e-11, 8.6435557e-06, 1.82597406e-20, 1.23365967e-36, 6.46420625e-11, 4.13620068e-22, 4.14639775e-26, 9.43163451e-21, 5.10727563e-21, 4.89207359e-08, 1.70855396e-18, 9.58663e-10, 8.14078749e-10, 1.96881183e-29, 3.65758581e-07, 1.31264777e-10, 8.82629747e-10, 3.49284174e-19, 1.56340012e-20, 0.0201981962, 5.43972e-27, 1.02918339e-18, 9.67634833e-07, 8.77562769e-08, 4.43145826e-10, 0.0155775016, 1.19980748e-09, 1.21633584e-12, 9.94060834e-10, 6.01758809e-10, 7.64296688e-18, 2.24728327e-14, 3.62570136e-31, 9.13382461e-20, 1.68064736e-26, 2.75453949e-05, 9.4986119e-10, 1.15438322e-16, 1.19038279e-30, 1.06983849e-18, 2.29192259e-17, 4.40645356e-24, 6.87184754e-16, 2.41873204e-05, 1.00572861e-09, 4.11976044e-11, 2.80309024e-07, 5.79043756e-21, 2.38929519e-16, 6.01525e-09, 3.12904607e-12, 5.9119349e-32, 3.28388141e-12, 5.40224476e-10, 2.76133694e-08, 2.00492523e-30, 2.79831939e-11, 9.86159487e-10, 8.99711097e-08, 8.89658338e-08, 8.74652628e-10, 6.32384678e-09, 5.9608858e-20, 0.0223438926, 1.95757133e-10, 3.58451e-18, 1.49573482e-14, 7.34355935e-13, 2.59326033e-11, 1.73901936e-17, 6.04357346e-17, 9.87296245e-10, 2.13098827e-09, 3.8833e-13, 1.18694405e-14, 2.21246268e-11, 4.99266503e-15, 1.10164378e-09, 1.22858408e-08, 1.8840988e-16, 3.95591681e-26, 8.1934222e-14, 5.60520798e-34, 7.44142503e-21, 3.6342778e-21, 4.8119432e-07, 5.96698302e-20, 2.95328317e-10, 0.00482289633, 5.42777953e-20, 1.89461105e-08, 8.97162e-10, 4.76586811e-27, 7.46943729e-10, 2.3949687e-18, 3.65442472e-27, 8.41880232e-10, 5.07921691e-11, 4.79976145e-22, 6.09665522e-29, 2.19439368e-20, 9.09918141e-10, 4.71256867e-10, 4.19200265e-11, 3.7205166e-11, 1.52387707e-22, 1.98937537e-38, 9.93875e-10, 9.00590935e-10, 2.84691177e-11, 3.83940232e-07, 4.05817784e-12, 1.42086158e-19, 1.66736144e-30, 4.34756717e-30, 2.70626675e-18, 4.03900826e-25, 2.58021061e-07, 6.57950626e-18, 4.28190061e-09, 6.13698748e-14, 8.99433861e-10, 3.9055787e-27, 7.30900834e-15, 2.53341673e-13, 5.26947e-13, 1.48395629e-10, 8.20232049e-10, 2.73083648e-24, 1.30961753e-09, 5.6977826e-14, 3.74509757e-09, 5.36615474e-11, 9.44967269e-07, 8.51479532e-28, 4.79679829e-06, 9.55144674e-09, 1.18474109e-14, 2.95350988e-09, 3.02194479e-07, 3.5272861e-12, 0.000454331544, 4.41743066e-11, 1.13462906e-10, 5.64344686e-13, 1.38149854e-14, 4.11394981e-11, 1.15306358e-11, 1.05700573e-10, 4.6891318e-21, 0.000430181157, 7.20089872e-07, 1.02017921e-07, 7.80676412e-09, 0.000342015293, 2.32318871e-06, 3.45381427e-12, 3.13393684e-05, 2.00994017e-12, 1.41756381e-20, 1.40913447e-16, 8.40106207e-10, 7.27618242e-15, 8.50766465e-14, 2.46819644e-17, 9.74116565e-10, 1.31892127e-11, 3.3643344e-16, 3.8013394e-13, 7.51644051e-18, 9.19359533e-10, 0.000123397389, 1.43491226e-11, 1.14407713e-15, 1.45158048e-18, 4.6589332e-15, 2.96256769e-11, 4.27737394e-11, 7.46379874e-06, 8.16309225e-08, 2.22868657e-09, 1.68678439e-11, 9.45632124e-08, 2.20742892e-23, 1.38563514e-27, 3.36360232e-24, 2.63495605e-27, 3.1305894e-10, 1.54527724e-24, 3.93448484e-19, 1.81882877e-15, 2.00288203e-17, 7.87939598e-08, 2.51006716e-19, 1.0006771e-09, 9.48637169e-10, 6.80443e-05, 1.04256581e-09, 2.56385954e-11, 1.97705449e-05, 1.88911087e-09, 5.25718195e-13, 0.00908518396, 4.36497626e-18, 1.96638785e-21, 8.61918943e-06, 1.80503978e-26, 1.68787972e-06, 3.30872271e-19, 2.18583707e-10, 4.09909485e-17, 2.7745541e-06, 9.29047186e-11, 2.52615034e-24, 3.50331507e-20, 4.44467076e-07, 1.56819471e-11, 3.7593191e-16, 8.92366403e-10, 1.56928068e-15, 1.37706579e-29, 1.45056461e-07, 1.29307538e-30, 3.87300185e-25, 3.96381841e-14, 1.08925e-27, 1.88884013e-17, 8.89067098e-10, 3.96512115e-13, 8.86665796e-10, 3.65795456e-15, 6.80242801e-21, 9.67297495e-25, 4.82866913e-10, 8.23007042e-07, 5.49229953e-22, 4.22410037e-18, 1.01920772e-09, 8.44768769e-11, 1.81603669e-24, 9.78185089e-13, 9.86761561e-10, 4.26121405e-13, 1.00592812e-09, 9.57473162e-07, 5.06824235e-11, 8.59884943e-20, 3.08035778e-05, 4.63198828e-30, 6.97826508e-09, 8.55942062e-26, 1.27705477e-16, 1.14443302e-26, 5.3115262e-25, 1.64871085e-06, 1.15059685e-14, 3.50077666e-20, 4.62180553e-22, 5.76208059e-09, 9.26635732e-32, 9.04760781e-29, 2.22944944e-11, 3.15071176e-17, 7.11442137e-16, 8.53464644e-06, 4.81154665e-23, 1.65776284e-26, 3.92500442e-11, 2.43079114e-07, 1.4106988e-17, 8.80574891e-10, 5.176802e-17, 4.84584107e-13, 3.51659507e-10, 0.000414148817, 4.86001284e-09, 1.71070434e-20, 8.62974e-27, 4.65964429e-15, 2.47502964e-15, 8.93030483e-10, 9.81728226e-18, 1.29627296e-08, 1.2003791e-10, 2.66500028e-15, 3.63831248e-07, 1.14567595e-22, 9.93238269e-10, 1.41469082e-15, 2.61589851e-26, 1.03670028e-09, 2.67973044e-19, 5.16726949e-18, 8.64574745e-10, 4.61441628e-14, 6.31998623e-21, 6.54846132e-13, 7.65624475e-14, 2.43329462e-27, 1.52661476e-18, 7.06949781e-20, 3.30569854e-13, 8.99979646e-10, 3.15000936e-09, 3.2660866e-26, 2.98960427e-15, 3.21170177e-17, 3.07165064e-05, 0.0321023799, 2.86876766e-05, 1.31885032e-23, 9.87260641e-17, 1.12977938e-09, 7.91978261e-10, 2.64420189e-14, 0.000765070727, 9.38354505e-10, 2.7072091e-31, 6.27555574e-24, 2.95824047e-05, 2.24707897e-24, 1.00366182e-11, 2.82120466e-10, 3.26466913e-17, 6.86079429e-16, 2.75170353e-09, 9.51709891e-12, 6.30056185e-09, 4.61919348e-12, 9.28752702e-23, 9.00779895e-10, 6.00785233e-06]]

In [38]:
next_word = np.argmax(predictions)
next_word

95

In [40]:
output_word = ""
for word, index in t.word_index.items():
      if index == next_word:
        output_word = word
        break

output_word


'feel'

In [41]:
import numpy as np

class LyricsPreprocessor(object):
  def __init__(self):
    print("Preprocessor initialized. This does nothing.")

  def preprocess(self, data):
    return data

In [42]:
import pickle

In [44]:
with open("preprocessor.pk1", "wb") as f:
  pickle.dump(LyricsPreprocessor(), f)

Preprocessor initialized. This does nothing.


In [6]:
m = 1

def set_max():
  global m
  m = 2

set_max()
m

2

In [8]:
import numpy as np
arr =  [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 19]]
arr = np.array(arr)

In [9]:
import google.protobuf
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

In [11]:
d = json_format.ParseDict(arr.tolist(), Value())
d 

list_value {
  values {
    list_value {
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        number_value: 0.0
      }
      values {
        numbe